In [ ]:
# %% Deep learning - Section 7.50
#    Linear solutions to linear problems

# This code pertains a deep learning course provided by Mike X. Cohen on Udemy:
#   > https://www.udemy.com/course/deeplearning_x
# The "base" code in this repository is adapted (with very minor modifications)
# from code developed by the course instructor (Mike X. Cohen), while the
# "exercises" and the "code challenges" contain more original solutions and
# creative input from my side. If you are interested in DL (and if you are
# reading this statement, chances are that you are), go check out the course, it
# is singularly good.


In [ ]:
# %% Libraries and modules
import numpy               as np
import matplotlib.pyplot   as plt
import torch
import torch.nn            as nn
import copy

from google.colab                     import files
from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('svg')


In [ ]:
# %% Data

nClust = 100
blur   = 1

A = [1, 3]
B = [1,-2]

a = [ A[0]+np.random.randn(nClust)*blur, A[1]+np.random.randn(nClust)*blur ]
b = [ B[0]+np.random.randn(nClust)*blur, B[1]+np.random.randn(nClust)*blur ]

# True labels
labels_np = np.vstack(( np.zeros((nClust,1)), np.ones((nClust,1)) ))

# Concatenate
data_np = np.hstack((a,b)).T

# Convert into torch tensor
data   = torch.tensor(data_np).float()
labels = torch.tensor(labels_np).float()


In [ ]:
# Plotting

fig = plt.figure(figsize=(7,7))

plt.plot( data[np.where(labels==0)[0],0],data[np.where(labels==0)[0],1],'bs' )
plt.plot( data[np.where(labels==1)[0],0],data[np.where(labels==1)[0],1],'ro' )
plt.title('Some binary data')
plt.xlabel('x1')
plt.ylabel('x2')

plt.savefig('figure54_linear_solutions_linear_problems.png')

plt.show()

files.download('figure54_linear_solutions_linear_problems.png')


In [ ]:
# %% Functions


In [ ]:
# %% Function to build the model

# Remove non-linear activation function from input and hidden layers, only keep
# the sigmoid at the end

def gen_model(learning_rate):

    # Model architecture (ReLUs inside, sigmoid at the end)
    ANNclassify = nn.Sequential(
                     nn.Linear(2,16),  # input layer (num inputs, num outputs)
                     #nn.ReLU(),        # first activation function
                     nn.Linear(16,1),  # hidden layer (num inputs, num outputs)
                     #nn.ReLU(),        # second activation function
                     nn.Linear(1,1),   # output layer (num inputs, num outputs)
                     nn.Sigmoid()      # third activation function (not needed if use BCEWithLogitsLoss)
                     )

    # Loss function
    loss_fun = nn.BCELoss()

    # Optimizer
    optimizer = torch.optim.SGD(ANNclassify.parameters(),lr=learning_rate)

    # Outputs
    return ANNclassify,loss_fun,optimizer


In [ ]:
# %% Function to train the model

num_epochs = 1000

def train_model(ANNmodel):

    # Initialise losses
    losses = torch.zeros(num_epochs)

    # Loop over epochs
    for epoch_i in range(num_epochs):

        # Forward propagation
        yHat = ANNmodel(data)

        # Compute loss
        loss = loss_fun(yHat,labels)
        losses[epoch_i] = loss

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Final forward pass
    predictions = ANNmodel(data)

    # Accuracy (here .5 because manual Sigmoid implementation [no BCEWithLogitsLoss])
    tot_acc = 100*torch.mean( ((predictions>.5)==labels).float() )

    return losses,predictions,tot_acc


In [ ]:
# %% Parametric experiment on learning rates

# Parameters
learning_rates   = np.linspace(.001,.1,50)
acc_by_learnRate = []
all_losses       = np.zeros( (len(learning_rates),num_epochs) )

# Loop through learning rates
for i,lr in enumerate(learning_rates):

    # Generate and run the model
    ANNclassify,lossfun,optimizer = gen_model(lr)
    losses,predictions,tot_acc    = train_model(ANNclassify)

    # Store results
    acc_by_learnRate.append(tot_acc)
    all_losses[i,:] = losses.detach()


In [ ]:
# Plotting

fig,ax = plt.subplots(1,2,figsize=(12,4))

ax[0].plot(learning_rates,acc_by_learnRate,'s-')
ax[0].set_xlabel('Learning rate')
ax[0].set_ylabel('Accuracy (mind the axis scale!)')
ax[0].set_title('Accuracy by learning rate')
ax[0].set_title('Accuracy by learning rate')

ax[1].plot(all_losses.T)
ax[1].set_title('Losses by learning rate')
ax[1].set_xlabel('Epoch number')
ax[1].set_ylabel('Loss (mind the axis scale!)')

plt.savefig('figure55_linear_solutions_linear_problems.png')

plt.show()

files.download('figure55_linear_solutions_linear_problems.png')
